<a href="https://colab.research.google.com/github/afriverat25/Futbol-Analatics/blob/main/FutbolAnalitics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==========================
# 1. Instalar dependencias
# ==========================
!pip install ultralytics supervision deep-sort-realtime opencv-python

# ==========================
# 2. Importar librerías
# ==========================
from ultralytics import YOLO
import cv2
from deep_sort_realtime.deepsort_tracker import DeepSort
# Requerimientos (ejecutar 1 vez)
# ----------------------------
!pip install -q ultralytics deep-sort-realtime opencv-python-headless==4.7.0.72 matplotlib
import random

# ----------------------------
# Imports y chequeos
# ----------------------------
import cv2
import numpy as np
import random
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import math
import time
from google.colab.patches import cv2_imshow  # solo si quieres mostrar imágenes puntuales en Colab

# Opcional: verificar GPU disponible para ultralytics/torch
import torch
print("CUDA disponible:", torch.cuda.is_available())
if torch.cuda.is_available():
    try:
        print("GPU:", torch.cuda.get_device_name(0))
    except:
        pass


# ==========================
# 3. Cargar modelo YOLOv8
# ==========================
# Puedes cambiar "yolov8n.pt" por "yolov8s.pt" si quieres algo más preciso
model = YOLO("yolov8s.pt")

# ==========================
# 4. Inicializar tracker
# ==========================
tracker = DeepSort(max_age=30)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 71.8 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 MB 9.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 2.0.8 requires opencv-python-headless>=4.9.0.80, but you have opencv-python-headless 4.7.0.72 which is incompatible.
albucore 0.0.24 requires opencv-python-headless>=4.9.0.80, but you hav

In [2]:
# ==========================
# 5. Abrir video de entrada
# ==========================
input_path = "/content/66.mp4"   # 👉 Carga del Video
cap = cv2.VideoCapture(input_path)

# Configurar video de salida
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("resultado.mp4", fourcc,
                      cap.get(cv2.CAP_PROP_FPS),
                      (int(cap.get(3)), int(cap.get(4))))

In [3]:

# ----------------------------
# Paths / parámetros
# ----------------------------
VIDEO_PATH = "/content/10.mp4"   # tu video
OUTPUT_PATH = "resultado_filtrado.mp4"
YOLO_WEIGHTS = "yolov8s.pt"

FIELD_METERS = (105, 68)
FIELD_PX_PER_M = 10
FIELD_W_PX = int(FIELD_METERS[0] * FIELD_PX_PER_M)
FIELD_H_PX = int(FIELD_METERS[1] * FIELD_PX_PER_M)

SMOOTH_ALPHA = 0.35
SKIP_FRAMES = 1

# ----------------------------
# Modelo y tracker
# ----------------------------
model = YOLO(YOLO_WEIGHTS)
tracker = DeepSort(max_age=30)

cap = cv2.VideoCapture(VIDEO_PATH)
frame_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) or 20.0

scale_field = frame_h / FIELD_H_PX
field_vis_w = int(FIELD_W_PX * scale_field)
field_vis_h = frame_h
out_w = frame_w + field_vis_w
out_h = frame_h

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps, (out_w, out_h))

# ----------------------------
# Helpers
# ----------------------------
colors = {}
last_pos = {}

def get_color(track_id):
    if track_id not in colors:
        colors[track_id] = (random.randint(40,230), random.randint(40,230), random.randint(40,230))
    return colors[track_id]

def smooth_pos(track_id, x, y):
    if track_id not in last_pos:
        last_pos[track_id] = (x, y)
        return x, y
    lx, ly = last_pos[track_id]
    nx = int((1 - SMOOTH_ALPHA) * lx + SMOOTH_ALPHA * x)
    ny = int((1 - SMOOTH_ALPHA) * ly + SMOOTH_ALPHA * y)
    last_pos[track_id] = (nx, ny)
    return nx, ny

def pixel_to_field(cx, cy, frame_w, frame_h, field_w_px, field_h_px):
    fx = cx / frame_w * field_w_px
    fy = cy / frame_h * field_h_px
    return int(fx), int(fy)

# ----------------------------
# Clasificación uniforme por color (HSV más flexible)
# ----------------------------
def classify_team(frame, box):
    x1, y1, x2, y2 = map(int, box)
    crop = frame[y1:y2, x1:x2]
    if crop.size == 0:
        return "otro"
    hsv = cv2.cvtColor(crop, cv2.COLOR_BGR2HSV)

    # Máscaras
    mask_white = cv2.inRange(hsv, (0,0,160), (180,80,255))
    mask_red1 = cv2.inRange(hsv, (0,100,80), (10,255,255))
    mask_red2 = cv2.inRange(hsv, (160,100,80), (180,255,255))
    mask_red = cv2.bitwise_or(mask_red1, mask_red2)
    mask_yellow = cv2.inRange(hsv, (18,80,80), (35,255,255))

    total = crop.shape[0] * crop.shape[1]
    white_ratio = np.sum(mask_white) / total
    red_ratio = np.sum(mask_red) / total
    yellow_ratio = np.sum(mask_yellow) / total

    if white_ratio > 0.15 and white_ratio > red_ratio and white_ratio > yellow_ratio:
        return "blanco"
    elif red_ratio > 0.15 and red_ratio > white_ratio and red_ratio > yellow_ratio:
        return "rojo"
    elif yellow_ratio > 0.08:  # umbral más laxo
        return "amarillo"
    else:
        return "otro"

# ----------------------------
# Detección círculo central
# ----------------------------
def detectar_circulo_central(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.medianBlur(gray, 5)
    circles = cv2.HoughCircles(
        gray, cv2.HOUGH_GRADIENT, dp=1.2, minDist=100,
        param1=50, param2=30, minRadius=20, maxRadius=100
    )
    if circles is not None:
        circles = np.uint16(np.around(circles))
        return circles[0][0]  # (cx, cy, r)
    return None

# ----------------------------
# Helpers de cancha
# ----------------------------
def detect_field_lines(frame):
    """
    Detecta las líneas del campo usando Canny + Hough.
    Devuelve una lista de segmentos de línea.
    """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    edges = cv2.Canny(blur, 50, 150, apertureSize=3)

    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=120,
                            minLineLength=80, maxLineGap=10)
    if lines is None:
        return []
    return lines[:,0]  # (x1,y1,x2,y2)


def get_court_keypoints(lines):
    """
    A partir de las líneas detectadas intenta obtener puntos de la cancha.
    Para simplificar: devuelve esquinas aproximadas (solo si hay suficientes líneas).
    """
    if len(lines) < 4:
        return None

    # Extraer todos los extremos
    pts = []
    for (x1,y1,x2,y2) in lines:
        pts.append((x1,y1))
        pts.append((x2,y2))

    # Convex hull para aproximar contorno de la cancha
    pts = np.array(pts)
    hull = cv2.convexHull(pts)
    if len(hull) < 4:
        return None

    # Tomamos 4 puntos principales (esquinas del hull)
    hull = hull.squeeze()
    rect = cv2.boundingRect(hull)
    x,y,w,h = rect
    approx_court = np.array([
        [x,y], [x+w,y],
        [x+w,y+h], [x,y+h]
    ], dtype=np.float32)

    return approx_court


def get_reference_court():
    """
    Define el rectángulo de referencia del campo en el plano 2D (homografía destino).
    """
    return np.array([
        [0,0],
        [FIELD_W_PX,0],
        [FIELD_W_PX,FIELD_H_PX],
        [0,FIELD_H_PX]
    ], dtype=np.float32)

def pixel_to_field_homography(x, y, H):
    """
    Transforma un punto (x,y) de la imagen al plano del campo usando homografía H.
    """
    pt = np.array([[x,y]], dtype=np.float32)
    pt = np.array([pt])
    dst = cv2.perspectiveTransform(pt, H)
    return int(dst[0,0,0]), int(dst[0,0,1])

# ===============================
# 1. Detectar líneas del campo
# ===============================
def detectar_lineas_campo(frame):
    """
    Detecta líneas blancas en la cancha usando Canny + HoughLinesP.
    Devuelve los segmentos detectados.
    """
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, (0, 0, 200), (180, 40, 255))  # rango blanco
    edges = cv2.Canny(mask, 50, 150)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=100, minLineLength=80, maxLineGap=10)
    return lines

# ===============================
# 2. Calcular homografía
# ===============================
def calcular_homografia(puntos_detectados, puntos_campo):
    """
    Calcula la matriz de homografía que transforma puntos detectados -> puntos del campo 2D.
    """
    H, _ = cv2.findHomography(puntos_detectados, puntos_campo, cv2.RANSAC, 5.0)
    return H

# ===============================
# 3. Dibujar cancha en 2D
# ===============================
def draw_field(canvas):
    h, w = canvas.shape[:2]
    # Fondo verde
    canvas[:] = (0, 128, 0)

    # Línea de medio campo
    cv2.line(canvas, (w//2, 0), (w//2, h), (255, 255, 255), 2)

    # Círculo central
    cv2.circle(canvas, (w//2, h//2), 60, (255, 255, 255), 2)

    # Áreas grandes
    cv2.rectangle(canvas, (0, h//3), (80, h*2//3), (255, 255, 255), 2)          # Izq
    cv2.rectangle(canvas, (w-80, h//3), (w, h*2//3), (255, 255, 255), 2)        # Der

    # Áreas chicas
    cv2.rectangle(canvas, (0, h//2-40), (30, h//2+40), (255, 255, 255), 2)      # Izq
    cv2.rectangle(canvas, (w-30, h//2-40), (w, h//2+40), (255, 255, 255), 2)    # Der

    return canvas


def dibujar_campo(w, h):
    canvas = np.zeros((h, w, 3), dtype=np.uint8)
    return draw_field(canvas)

# ===============================
# 4. Proyectar detecciones
# ===============================
def proyectar_jugadores(detecciones, H, campo):
    """
    Proyecta las posiciones de los jugadores sobre el campo en 2D.
    """
    for (x, y, w, h) in detecciones:
        cx, cy = x + w // 2, y + h // 2
        punto = np.array([[cx, cy]], dtype="float32")
        punto = np.array([punto])
        dst = cv2.perspectiveTransform(punto, H)
        px, py = int(dst[0][0][0]), int(dst[0][0][1])

        if 0 <= px < campo.shape[1] and 0 <= py < campo.shape[0]:
            cv2.circle(campo, (px, py), 5, (0, 0, 255), -1)



def filtrar_superposiciones(detecciones, umbral_iou=0.5):
    """
    detecciones = lista de [x1, y1, x2, y2, conf, clase]
    """
    if len(detecciones) == 0:
        return []

    boxes = [d[:4] for d in detecciones]
    confs = [d[4] for d in detecciones]
    clases = [d[5] for d in detecciones]

    indices = cv2.dnn.NMSBoxes(boxes, confs, score_threshold=0.3, nms_threshold=umbral_iou)

    filtradas = []
    if len(indices) > 0:
        for i in indices.flatten():
            filtradas.append(detecciones[i])
    return filtradas

# ----------------------------
# Bucle principal
# ----------------------------
# ----------------------------
# Bucle principal
# ----------------------------
MIN_AREA = 500      # evita público
MAX_AREA = 80000    # evita repeticiones grandes
IOU_THRESHOLD = 0.5 # umbral para considerar dos detecciones como duplicadas

frame_id = 0
circulo_central = None
start_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_id += 1

    annotated = frame.copy()
    detections_for_tracker = []

    # ----------------------------
    # 1) Detectar líneas de cancha y homografía
    # ----------------------------
    field_lines = detect_field_lines(frame)
    court_points = get_court_keypoints(field_lines)

    if court_points is None:
        print(f"Frame {frame_id}: sin cancha → ignorado")
        continue

    H, _ = cv2.findHomography(court_points, get_reference_court())
    if H is None:
        print(f"Frame {frame_id}: sin homografía → ignorado")
        continue

    # ----------------------------
    # 2) Detectar círculo central (solo una vez)
    # ----------------------------
    if circulo_central is None:
        circulo_central = detectar_circulo_central(frame)
        if circulo_central is not None:
            cx_circ, cy_circ, r_circ = circulo_central
            cv2.circle(annotated, (cx_circ, cy_circ), r_circ, (0,255,0), 2)

    # ----------------------------
    # 3) YOLO detecciones + filtrado
    # ----------------------------
    raw_dets = []
    results = model(frame, conf=0.35, verbose=False)[0]
    for box in results.boxes:
        cls = int(box.cls[0])
        label = model.names[cls]
        if label not in ["person", "sports ball"]:
            continue
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf[0])
        w, h = x2 - x1, y2 - y1
        area = w * h
        if area < MIN_AREA or area > MAX_AREA:
            continue
        raw_dets.append([x1, y1, x2, y2, conf, label])

    # 🔹 aplicar NMS para evitar duplicados
    raw_dets = filtrar_superposiciones(raw_dets, IOU_THRESHOLD)

    for (x1, y1, x2, y2, conf, label) in raw_dets:
        detections_for_tracker.append([[x1, y1, x2-x1, y2-y1], conf, label])

    # ----------------------------
    # 4) Tracking
    # ----------------------------
    tracks = tracker.update_tracks(detections_for_tracker, frame=frame)

    # Crear campo verde con líneas
    field_vis = dibujar_campo(field_vis_w, field_vis_h)

    # ----------------------------
    # 5) Procesar cada track
    # ----------------------------
    for track in tracks:
        if not track.is_confirmed():
            continue
        l, t, r, b = track.to_ltrb()
        track_id = track.track_id
        label = track.get_det_class()

        cx, cy = int((l+r)/2), int((t+b)/2)
        sx, sy = smooth_pos(track_id, cx, cy)

        if label == "person":
            team = classify_team(frame, (l,t,r,b))
            if team == "otro":
                continue

            # Mitad de cancha
            if circulo_central is not None:
                cx_circ, cy_circ, r_circ = circulo_central
                lado = "izq" if (l+r)/2 < cx_circ else "der"
                team = f"{team}-{lado}"

            if "blanco" in team: col = (255,255,255)
            elif "rojo" in team: col = (0,0,255)
            elif "amarillo" in team: col = (0,255,255)
            else: continue

        elif label == "sports ball":
            team = "ball"
            col = (0,165,255)
        else:
            continue

        # --- Dibujo en video ---
        cv2.rectangle(annotated, (int(l), int(t)), (int(r), int(b)), col, 2)
        cv2.putText(annotated, f"{team}-{track_id}", (int(l), int(t)-8),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, col, 2)
        cv2.circle(annotated, (sx, sy), 4, col, -1)

        # --- Dibujo en campo 2D usando homografía ---
        fx, fy = pixel_to_field_homography(sx, sy, H)
        fx_vis = int(fx * scale_field)
        fy_vis = int(fy * scale_field)
        cv2.circle(field_vis, (fx_vis, fy_vis), 10 if team!="ball" else 6, col, -1)

    # ----------------------------
    # 6) Combinar video + campo
    # ----------------------------
    combined = np.zeros((out_h, out_w, 3), dtype=np.uint8)
    combined[:, :frame_w] = annotated
    combined[:, frame_w:] = field_vis
    out.write(combined)

cap.release()
out.release()
elapsed = time.time() - start_time
print(f"✅ Terminado. Video de salida: {OUTPUT_PATH}  (tiempo: {elapsed:.1f}s)")



Frame 3: sin cancha → ignorado
Frame 434: sin cancha → ignorado
Frame 435: sin cancha → ignorado
Frame 436: sin cancha → ignorado
Frame 449: sin cancha → ignorado
Frame 450: sin cancha → ignorado
Frame 451: sin cancha → ignorado
Frame 452: sin cancha → ignorado
Frame 453: sin cancha → ignorado
Frame 455: sin cancha → ignorado
Frame 457: sin cancha → ignorado
Frame 459: sin cancha → ignorado
Frame 461: sin cancha → ignorado
Frame 463: sin cancha → ignorado
Frame 464: sin cancha → ignorado
Frame 468: sin cancha → ignorado
Frame 469: sin cancha → ignorado
Frame 470: sin cancha → ignorado
Frame 472: sin cancha → ignorado
Frame 479: sin cancha → ignorado
Frame 480: sin cancha → ignorado
Frame 482: sin cancha → ignorado
Frame 483: sin cancha → ignorado
Frame 484: sin cancha → ignorado
Frame 485: sin cancha → ignorado
Frame 486: sin cancha → ignorado
Frame 487: sin cancha → ignorado
Frame 488: sin cancha → ignorado
Frame 489: sin cancha → ignorado
Frame 490: sin cancha → ignorado
Frame 491: s